## Verifying nearest distance to reciever feature

In [1]:
import pandas as pd

plays = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/plays.csv")

distance_feature = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/nearest_defender_at_snap.csv")

In [2]:
print(distance_feature.head())

   Unnamed: 0      gameId  playId  nearest_defender_1  nearest_defender_2  \
0           0  2022091107     301                4.26                4.59   
1          23  2022091113    3642                2.19                0.00   
2          46  2022091111    2257                2.91                2.54   
3          69  2022091107    3864                5.88                1.93   
4          92  2022091105    3000                6.71                4.89   

   nearest_defender_3  nearest_defender_4  nearest_defender_5  
0                7.91                 0.0                 0.0  
1                0.00                 0.0                 0.0  
2                3.63                 0.0                 0.0  
3                3.01                 0.0                 0.0  
4                6.11                 0.0                 0.0  


In [3]:
import pandas as pd

defender_cols = [col for col in distance_feature.columns if col.startswith("nearest_defender_")]

distance_feature['maxDistance'] = distance_feature[defender_cols].max(axis=1)

distance_feature['averageDistance'] = (
    distance_feature[defender_cols]
    .replace(0, pd.NA) 
    .mean(axis=1, skipna=True)
)

defender_columns = ['nearest_defender_1', 'nearest_defender_2', 'nearest_defender_3', 'nearest_defender_4', 'nearest_defender_5']

# Create a new column for the total sum
distance_feature['total_nearest_defender_distance'] = distance_feature[defender_columns].sum(axis=1)

distance_feature['numReceivers'] = (distance_feature[defender_cols] != 0).sum(axis=1)

result = distance_feature[['gameId', 'playId', 'maxDistance', 'averageDistance', 'numReceivers', 'total_nearest_defender_distance']]
result['averageDistance'] = pd.to_numeric(result['averageDistance'], errors='coerce')

result.describe()



/tmp/ipykernel_3841439/1130996696.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['averageDistance'] = pd.to_numeric(result['averageDistance'], errors='coerce')


,gameId,playId,maxDistance,averageDistance,numReceivers,total_nearest_defender_distance
count,1.612400e+04,16124.000000,16124.000000,16044.000000,16124.000000,16124.000000
mean,2.022099e+09,2023.830563,6.686266,5.204801,2.539072,13.213761
std,5.979252e+03,1182.090771,2.505291,1.877748,0.689650,5.837911
min,2.022091e+09,54.000000,0.000000,1.280000,0.000000,0.000000
25%,2.022093e+09,996.000000,4.750000,3.763333,2.000000,9.070000
50%,2.022101e+09,2017.000000,6.600000,4.980000,3.000000,12.575000
75%,2.022102e+09,3022.000000,8.540000,6.373333,3.000000,17.010000
max,2.022111e+09,5120.000000,21.800000,18.550000,5.000000,47.360000


In [4]:
plays2 = plays[["gameId", "playId","expectedPointsAdded","yardsGained"]]

dist_rec = plays2.merge(result, on=['gameId', 'playId'], how='inner')

dist_rec['bigYardPlay'] = (dist_rec['yardsGained'] > 15).astype(int)
dist_rec['bigEpaPlay'] = (dist_rec['expectedPointsAdded'] > 1.5).astype(int)

dist_rec.describe()

,gameId,playId,expectedPointsAdded,yardsGained,maxDistance,averageDistance,numReceivers,total_nearest_defender_distance,bigYardPlay,bigEpaPlay
count,1.612400e+04,16124.000000,16124.000000,16124.000000,16124.000000,16044.000000,16124.000000,16124.000000,16124.000000,16124.000000
mean,2.022099e+09,2023.830563,-0.029381,5.460618,6.686266,5.204801,2.539072,13.213761,0.098859,0.110146
std,5.979252e+03,1182.090771,1.397405,8.833095,2.505291,1.877748,0.689650,5.837911,0.298482,0.313082
min,2.022091e+09,54.000000,-13.023600,-68.000000,0.000000,1.280000,0.000000,0.000000,0.000000,0.000000
25%,2.022093e+09,996.000000,-0.645440,0.000000,4.750000,3.763333,2.000000,9.070000,0.000000,0.000000
50%,2.022101e+09,2017.000000,-0.178783,3.000000,6.600000,4.980000,3.000000,12.575000,0.000000,0.000000
75%,2.022102e+09,3022.000000,0.696452,8.000000,8.540000,6.373333,3.000000,17.010000,0.000000,0.000000
max,2.022111e+09,5120.000000,8.698986,98.000000,21.800000,18.550000,5.000000,47.360000,1.000000,1.000000


#### Looking at max distance

In [5]:
variable = 'maxDistance'

q1_dist = dist_rec[variable].quantile(0.25)
q3_dist = dist_rec[variable].quantile(0.75)

dist_low = dist_rec[dist_rec[variable] <= q1_dist]
dist_mid = dist_rec[(dist_rec[variable] > q1_dist) & 
                        (dist_rec[variable] <= q3_dist)]
dist_high = dist_rec[dist_rec[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")

top_20 = dist_rec.nlargest(50, variable)
bottom_50 = dist_rec.nsmallest(50, variable)

print_group_stats(bottom_50 , "bottom50")
print_group_stats(top_20, "top50")


variable < 25% — EPA Mean: -0.014, Yards Gained Mean: 5.02, BigYards: 0.09, BigEPA: 0.12
variable 25%-75% — EPA Mean: -0.024, Yards Gained Mean: 5.58, BigYards: 0.10, BigEPA: 0.11
variable > 75% — EPA Mean: -0.055, Yards Gained Mean: 5.67, BigYards: 0.10, BigEPA: 0.10
bottom50 — EPA Mean: 0.260, Yards Gained Mean: 1.90, BigYards: 0.02, BigEPA: 0.08
top50 — EPA Mean: -0.133, Yards Gained Mean: 4.78, BigYards: 0.10, BigEPA: 0.02


In [13]:
def get_sample_plays(group, n=20):
    sample = group.sample(n=n, random_state=42)
    return [[int(row['gameId']), int(row['playId'])] for _, row in sample.iterrows()]

low_sample = get_sample_plays(dist_low)
mid_sample = get_sample_plays(dist_mid)
high_sample = get_sample_plays(dist_high)

all_samples = low_sample + mid_sample + high_sample

print(all_samples)


[[2022100204, 56], [2022100207, 3048], [2022091811, 2937], [2022091113, 2009], [2022103007, 3750], [2022100206, 2135], [2022091103, 2243], [2022091110, 400], [2022091104, 3016], [2022101602, 402], [2022100208, 3654], [2022091808, 2970], [2022101300, 896], [2022102000, 1104], [2022100912, 2936], [2022110603, 87], [2022091900, 3302], [2022100300, 3793], [2022092506, 1941], [2022101606, 3606], [2022091812, 2601], [2022092509, 1602], [2022103009, 100], [2022101602, 2161], [2022100205, 3839], [2022091109, 2803], [2022102000, 2633], [2022103002, 1399], [2022092506, 3908], [2022100204, 633], [2022102303, 2741], [2022091901, 3918], [2022091108, 947], [2022091107, 2161], [2022110605, 1796], [2022102306, 3168], [2022092900, 3198], [2022100207, 1202], [2022100211, 1616], [2022103011, 728], [2022091807, 212], [2022110609, 3714], [2022091800, 1642], [2022100904, 1861], [2022102301, 2667], [2022091804, 2826], [2022110607, 649], [2022103000, 2274], [2022091112, 379], [2022101611, 3615], [2022102302, 

#### Looking at average distance

In [6]:
variable = 'averageDistance'

q1_dist = dist_rec[variable].quantile(0.25)
q3_dist = dist_rec[variable].quantile(0.75)

dist_low = dist_rec[dist_rec[variable] <= q1_dist]
dist_mid = dist_rec[(dist_rec[variable] > q1_dist) & 
                        (dist_rec[variable] <= q3_dist)]
dist_high = dist_rec[dist_rec[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")

top_20 = dist_rec.nlargest(50, variable)
bottom_50 = dist_rec.nsmallest(50, variable)

print_group_stats(bottom_50 , "bottom50")
print_group_stats(top_20, "top50")

variable < 25% — EPA Mean: -0.017, Yards Gained Mean: 5.15, BigYards: 0.10, BigEPA: 0.13
variable 25%-75% — EPA Mean: -0.028, Yards Gained Mean: 5.56, BigYards: 0.10, BigEPA: 0.11
variable > 75% — EPA Mean: -0.049, Yards Gained Mean: 5.64, BigYards: 0.10, BigEPA: 0.09
bottom50 — EPA Mean: -0.038, Yards Gained Mean: 4.00, BigYards: 0.06, BigEPA: 0.12
top50 — EPA Mean: -0.050, Yards Gained Mean: -0.04, BigYards: 0.02, BigEPA: 0.00


In [12]:
def get_sample_plays(group, n=20):
    sample = group.sample(n=n, random_state=40)
    return [[int(row['gameId']), int(row['playId'])] for _, row in sample.iterrows()]

low_sample = get_sample_plays(dist_low)
mid_sample = get_sample_plays(dist_mid)
high_sample = get_sample_plays(dist_high)

all_samples2 = low_sample + mid_sample + high_sample

print(all_samples)


[[2022100909, 1483], [2022100911, 3022], [2022091103, 1126], [2022092510, 414], [2022103100, 1258], [2022110602, 2849], [2022100209, 3196], [2022101607, 3572], [2022100905, 193], [2022102308, 3280], [2022091102, 2373], [2022100901, 1004], [2022091805, 3445], [2022110607, 2367], [2022092506, 3997], [2022101604, 975], [2022102311, 256], [2022091107, 4071], [2022091805, 3824], [2022092600, 255], [2022091101, 850], [2022091803, 2243], [2022100203, 1773], [2022103005, 4504], [2022100900, 1892], [2022102311, 278], [2022103009, 2964], [2022091804, 1097], [2022092200, 982], [2022101602, 2861], [2022110610, 170], [2022092504, 2771], [2022100904, 1247], [2022110610, 348], [2022100903, 619], [2022091113, 1566], [2022092507, 721], [2022100203, 2237], [2022103005, 1788], [2022103012, 2900], [2022100300, 3129], [2022110600, 2703], [2022103005, 1743], [2022092510, 2097], [2022100908, 3146], [2022102308, 3141], [2022110602, 3106], [2022100204, 289], [2022091805, 1243], [2022091809, 1360], [2022100208,

In [7]:
variable = 'averageDistance'

q1_dist = dist_rec[variable].quantile(0.25)
q3_dist = dist_rec[variable].quantile(0.75)

dist_low = dist_rec[dist_rec[variable] <= q1_dist]
dist_mid = dist_rec[(dist_rec[variable] > q1_dist) & 
                        (dist_rec[variable] <= q3_dist)]
dist_high = dist_rec[dist_rec[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")

top_20 = dist_rec.nlargest(50, variable)
bottom_50 = dist_rec.nsmallest(50, variable)

print_group_stats(bottom_50 , "bottom50")
print_group_stats(top_20, "top50")

variable < 25% — EPA Mean: -0.017, Yards Gained Mean: 5.15, BigYards: 0.10, BigEPA: 0.13
variable 25%-75% — EPA Mean: -0.028, Yards Gained Mean: 5.56, BigYards: 0.10, BigEPA: 0.11
variable > 75% — EPA Mean: -0.049, Yards Gained Mean: 5.64, BigYards: 0.10, BigEPA: 0.09
bottom50 — EPA Mean: -0.038, Yards Gained Mean: 4.00, BigYards: 0.06, BigEPA: 0.12
top50 — EPA Mean: -0.050, Yards Gained Mean: -0.04, BigYards: 0.02, BigEPA: 0.00


In [14]:
not_confused_plays = all_samples + all_samples2
print(not_confused_plays)

[[2022100204, 56], [2022100207, 3048], [2022091811, 2937], [2022091113, 2009], [2022103007, 3750], [2022100206, 2135], [2022091103, 2243], [2022091110, 400], [2022091104, 3016], [2022101602, 402], [2022100208, 3654], [2022091808, 2970], [2022101300, 896], [2022102000, 1104], [2022100912, 2936], [2022110603, 87], [2022091900, 3302], [2022100300, 3793], [2022092506, 1941], [2022101606, 3606], [2022091812, 2601], [2022092509, 1602], [2022103009, 100], [2022101602, 2161], [2022100205, 3839], [2022091109, 2803], [2022102000, 2633], [2022103002, 1399], [2022092506, 3908], [2022100204, 633], [2022102303, 2741], [2022091901, 3918], [2022091108, 947], [2022091107, 2161], [2022110605, 1796], [2022102306, 3168], [2022092900, 3198], [2022100207, 1202], [2022100211, 1616], [2022103011, 728], [2022091807, 212], [2022110609, 3714], [2022091800, 1642], [2022100904, 1861], [2022102301, 2667], [2022091804, 2826], [2022110607, 649], [2022103000, 2274], [2022091112, 379], [2022101611, 3615], [2022102302, 